In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import Adam
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset = pd.read_csv("data.csv")

In [4]:
dataset.shape
dataset.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [5]:
X = dataset.drop(columns=['Unnamed: 0', 'y'], axis=1)
X.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
1,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73


In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

C:\Users\HCL\AppData\Local\Continuum\anaconda4\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\HCL\AppData\Local\Continuum\anaconda4\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[[ 0.88505134  1.20992878  1.46276429 ... -0.63414367 -0.43329036
  -0.23539922]
 [ 2.40057718  2.36619038  2.23944096 ...  1.02342937  0.95424076
   0.85653664]
 [-0.12328657 -0.16915405 -0.22513147 ... -0.13687176 -0.13859348
  -0.14440456]
 ...
 [ 0.1544592   0.10184476 -0.01720228 ...  0.0657205   0.07015014
   0.02545213]
 [-0.17159018 -0.08484331  0.00725997 ...  0.49546166  0.43852123
   0.40762968]
 [ 0.24502848  0.31262161  0.41088722 ...  0.0657205   0.08856869
   0.19530882]]


In [7]:
Y = dataset['y']
Y.head()

0    4
1    1
2    5
3    5
4    5
Name: y, dtype: int64

In [8]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
print(encoded_Y)

[3 0 4 ... 4 2 3]


In [9]:
categorical_Y = np_utils.to_categorical(encoded_Y)
print(categorical_Y)

[[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [10]:
def baseline_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=178, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(5, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [11]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=200, verbose=1)

In [12]:
kfold = KFold(n_splits=5, shuffle=True, random_state=3)

In [13]:
results = cross_val_score(estimator, X, categorical_Y, cv=kfold)
print('Accuracy: {}'.format(results.mean()))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
9200/9200 [==============================] - 10s 1ms/step - loss: 0.4209 - accuracy: 0.8146
Epoch 2/50
9200/9200 [==============================] - 5s 535us/step - loss: 0.3271 - accuracy: 0.8435
Epoch 3/50
9200/9200 [==============================] - 5s 537us/step - loss: 0.2944 - accuracy: 0.8536
Epoch 4/50
9200/9200 [==============================] - 5s 533us/step - loss: 0.2767 - accuracy: 0.8601
Epoch 5/50
9200/9200 [==============================] - 5s 533us/step - loss: 0.2672 - accuracy: 0.8644
Epoch 6/50
9200/9200 [==============================] - 5s 537us/step - loss: 0.2558 - accuracy: 0.8702
Epoch 7/50
9200/9200 [==============================] - 5s 533us/step - loss: 0.2474 - accuracy: 0.8757
Epoch 8/50
9200/9200 [==============================] - 4s 486us/step - loss: 0.2383 - accuracy: 0.8821
Epoch 9/50
9200/9200 [==============================] - 5s 493us/step - lo

Epoch 23/50
9200/9200 [==============================] - 5s 542us/step - loss: 0.1504 - accuracy: 0.9311
Epoch 24/50
9200/9200 [==============================] - 5s 546us/step - loss: 0.1429 - accuracy: 0.9368
Epoch 25/50
9200/9200 [==============================] - 5s 543us/step - loss: 0.1400 - accuracy: 0.9381
Epoch 26/50
9200/9200 [==============================] - 5s 549us/step - loss: 0.1350 - accuracy: 0.9398
Epoch 27/50
9200/9200 [==============================] - 5s 538us/step - loss: 0.1297 - accuracy: 0.9431
Epoch 28/50
9200/9200 [==============================] - 5s 564us/step - loss: 0.1257 - accuracy: 0.9447
Epoch 29/50
9200/9200 [==============================] - 5s 551us/step - loss: 0.1216 - accuracy: 0.9466
Epoch 30/50
9200/9200 [==============================] - 5s 539us/step - loss: 0.1210 - accuracy: 0.9467
Epoch 31/50
9200/9200 [==============================] - 5s 539us/step - loss: 0.1201 - accuracy: 0.9485
Epoch 32/50
9200/9200 [==============================] 

9200/9200 [==============================] - 4s 486us/step - loss: 0.1335 - accuracy: 0.9392
Epoch 28/50
9200/9200 [==============================] - 5s 528us/step - loss: 0.1301 - accuracy: 0.9429
Epoch 29/50
9200/9200 [==============================] - 5s 553us/step - loss: 0.1245 - accuracy: 0.9454
Epoch 30/50
9200/9200 [==============================] - 5s 492us/step - loss: 0.1217 - accuracy: 0.9467
Epoch 31/50
9200/9200 [==============================] - 4s 488us/step - loss: 0.1178 - accuracy: 0.9495
Epoch 32/50
9200/9200 [==============================] - 5s 491us/step - loss: 0.1140 - accuracy: 0.9499
Epoch 33/50
9200/9200 [==============================] - 4s 489us/step - loss: 0.1075 - accuracy: 0.9539
Epoch 34/50
9200/9200 [==============================] - 5s 492us/step - loss: 0.1077 - accuracy: 0.9540
Epoch 35/50
9200/9200 [==============================] - 5s 490us/step - loss: 0.1071 - accuracy: 0.9540
Epoch 36/50
9200/9200 [==============================] - 4s 487us/s

In [25]:
print('Accuracy: {}'.format(results))

Accuracy: [0.90182608 0.90330416 0.89791292 0.89895648 0.89634764]
